
# Clasificación multiclase — Diamonds (cut)

Objetivo: predecir **cut** (`Fair`, `Good`, `Very Good`, `Premium`, `Ideal`) con 5 modelos clásicos.

Pasos
1) Carga y exploración rápida  
2) Preprocesamiento (imputación + codificación con LabelEncoder + escalado numéricas)  
3) División train/test  
4) Entrenamiento: Regresión Logística, KNN, SVC, Árbol, Random Forest  
5) Evaluación: accuracy, matriz de confusión, classification_report y resumen final


In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

CSV_PATH = "/mnt/data/diamonds.csv"
TARGET = "cut"  # multiclase


In [ ]:

# 1) Carga y exploración rápida
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"No se encontró el dataset en {CSV_PATH}.")

df = pd.read_csv(CSV_PATH)

print("Dimensiones (filas, columnas):", df.shape)
print("Columnas:", df.columns.tolist())
print("\nTipos de datos:")
print(df.dtypes)

print("\nPrimeras 5 filas:")
display(df.head())

print("\nNulos por columna:")
print(df.isna().sum())

# Si el dataset es muy grande, permite muestrear (opcional)
MAX_ROWS = 5000
if len(df) > MAX_ROWS:
    df = df.sample(n=MAX_ROWS, random_state=42).reset_index(drop=True)
    print(f"\nSe tomó una muestra de {MAX_ROWS} filas para agilizar el entrenamiento.")


In [ ]:

# 2) Separar X (features) y y (objetivo)
if TARGET not in df.columns:
    raise ValueError(f"No encuentro la columna objetivo '{TARGET}'.")

feature_cols = [c for c in df.columns if c != TARGET]
X = df[feature_cols].copy()
y = df[TARGET].copy()

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

print("Numéricas:", num_cols)
print("Categóricas:", cat_cols)
print("Clases en y:", y.unique())


In [ ]:

# 3) Preprocesamiento explícito (sin Pipeline)

# Imputación numéricas -> media
num_imputer = SimpleImputer(strategy="mean")
X[num_cols] = num_imputer.fit_transform(X[num_cols])

# Imputación categóricas -> moda
cat_imputer = SimpleImputer(strategy="most_frequent")
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

# LabelEncoder en cada categórica de X
encoders_X = {}
for c in cat_cols:
    le = LabelEncoder()
    X[c] = le.fit_transform(X[c].astype(str))
    encoders_X[c] = le

# LabelEncoder para y (target multiclase)
le_y = LabelEncoder()
y_enc = le_y.fit_transform(y.astype(str))

# Estandarizar SOLO las numéricas
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

print("Preprocesamiento listo. Ejemplo de X preprocesado:")
display(X.head())
print("Clases codificadas (y):", dict(zip(le_y.classes_, range(len(le_y.classes_)))))


In [ ]:

# 4) División train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42, stratify=y_enc
)
print("Shapes -> X_train:", X_train.shape, "| X_test:", X_test.shape)


In [ ]:

# 5) Modelos a comparar
models = {
    "LogisticRegression": LogisticRegression(max_iter=200, multi_class="auto"),
    "KNN": KNeighborsClassifier(n_neighbors=7),
    "SVC": SVC(kernel="rbf"),
    "DecisionTree": DecisionTreeClassifier(random_state=42, max_depth=None),
    "RandomForest": RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
}
list(models.keys())


In [ ]:

# 6) Entrenamiento
fitted = {}
for name, clf in models.items():
    clf.fit(X_train, y_train)
    fitted[name] = clf
    print(f"{name}: entrenado.")


In [ ]:

def evaluar_modelo(name, clf, X_test, y_test, label_decoder=None):
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1m = f1_score(y_test, y_pred, average="macro")
    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.4f} | F1-macro: {f1m:.4f}")
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    print("Matriz de confusión:\n", cm)
    
    # Dibujo matriz
    plt.figure()
    plt.imshow(cm, interpolation="nearest")
    plt.title(f"Matriz de confusión — {name}")
    plt.colorbar()
    classes = label_decoder.classes_ if label_decoder is not None else np.arange(cm.shape[0]).astype(str)
    ticks = np.arange(len(classes))
    plt.xticks(ticks, classes, rotation=45, ha="right")
    plt.yticks(ticks, classes)
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, str(cm[i, j]), ha="center", va="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.xlabel("Predicción"); plt.ylabel("Real")
    plt.tight_layout()
    plt.show()
    
    # Reporte detallado
    print("Classification report:")
    print(classification_report(
        y_test, y_pred, zero_division=0,
        target_names=label_decoder.classes_ if label_decoder is not None else None
    ))
    return acc, f1m

# Evaluación de todos y resumen
results = {}
for name, clf in fitted.items():
    acc, f1m = evaluar_modelo(name, clf, X_test, y_test, label_decoder=le_y)
    results[name] = {{ "accuracy": acc, "f1_macro": f1m }}

print("\nResumen de métricas (ordenado por F1-macro):")
res_df = pd.DataFrame(results).T.sort_values(by=["f1_macro","accuracy"], ascending=False)
display(res_df)
best = res_df.index[0]
print(f"\nMejor modelo por F1-macro: {{best}} ->", res_df.loc[best].to_dict())
